# QUEST Example: Finding Argo and ICESat-2 data

In this notebook, we are going to find Argo and ICESat-2 data over a region of the Pacific Ocean. Normally, we would require multiple data portals or Python packages to accomplish this. However, thanks to the QUEST module, we can use icepyx to find both!

In [ ]:
# Basic packages
import geopandas as gpd
import h5py
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
from pprint import pprint
import requests

# icepyx and QUEST
import icepyx as ipx

## Define the Quest Object

The key to using icepyx for multiple datasets is to use the QUEST module. QUEST builds off of the general querying process originally designed for ICESat-2, but makes it applicable to other datasets.

Just like the ICESat-2 Query object, we begin by defining our Quest object. We provide the following bounding parameters:
* `spatial_extent`: Data is constrained to the given box over the Pacific Ocean.
* `date_range`: Only grab data from April 12-26, 2022.

In [ ]:
# Spatial bounds, given as SW/NE corners
spatial_extent = [-154, 30, -143, 37]

# Start and end dates, in YYYY-MM-DD format
date_range = ['2022-04-12', '2022-04-26']

# Initialize the QUEST object
reg_a = ipx.Quest(spatial_extent=spatial_extent, date_range=date_range)

print(reg_a)

Notice that we have defined our spatial and temporal domains, but we do not have any datasets in our QUEST object. The next section leads us through that process.

## Getting the data

Let's first grab the ICESat-2 data. If we want to extract information about the water column, the ATL03 product is likely the desired choice.
* `short_name`: ATL03 data only

In [ ]:
# ICESat-2 product
short_name = 'ATL03'

# Add ICESat-2 to QUEST datasets
reg_a.add_icesat2(product=short_name)
print(reg_a)

Let's see the available files over this region.

In [ ]:
pprint(reg_a.datasets['icesat2'].avail_granules(ids=True))

Note that many of the ICESat-2 functions shown here are the same as those used for normal icepyx queries. The user is referred to other examples for detailed explanations about other icepyx features.

Downloading ICESat-2 data requires Earthdata login credentials, and the `download_all()` function below gives an authentication check when attempting to access the ICESat-2 files.

Now let's grab Argo data using the same constraints. This is as simple as using the below function.

In [ ]:
# Add argo to the desired QUEST datasets
reg_a.add_argo()

Now we can access the data for both Argo and ICESat-2! The below function will do this for us.

**Important**: With our current code, the Argo data will be compiled into a Pandas DataFrame, which must be manually saved by the user. The ICESat-2 data is saved as processed HDF-5 files to the directory given below.

In [ ]:
# Access Argo and ICESat-2 data simultaneously
reg_a.download_all('/home/jovyan/icesat2-snowex/icepyx/quest-test-data/')

If the code worked correctly, then there should be 19 available Argo profiles, each containing `temperature` and `pressure`, compiled into a Pandas DataFrame. When BGC Argo is fully implemented to QUEST, we could add more variables to this list.

We also have a series of files containing ICESat-2 ATL03 data. Because these data files are very large, we are only going to focus on one of these files for this example.

Let's now load one of the ICESat-2 files and see where it passes relative to the Argo float data.

In [ ]:
# Load ICESat-2 latitudes, longitudes, heights, and photon confidence (optional)
path_root = '/home/jovyan/icesat2-snowex/icepyx/quest-test-data/'

is2_pd = pd.DataFrame()
with h5py.File(f'{path_root}processed_ATL03_20220419002753_04111506_006_02.h5', 'r') as f:
    is2_pd['lat'] = f['gt2l/heights/lat_ph'][:]
    is2_pd['lon'] = f['gt2l/heights/lon_ph'][:]
    is2_pd['height'] = f['gt2l/heights/h_ph'][:]
    is2_pd['signal_conf'] = f['gt2l/heights/signal_conf_ph'][:,1]
    
# Set Argo data as its own DataFrame
argo_df = reg_a.datasets['argo'].argodata

In [ ]:
# Convert both DataFrames into GeoDataFrames
is2_gdf = gpd.GeoDataFrame(is2_pd, 
                           geometry=gpd.points_from_xy(is2_pd.lon, is2_pd.lat),
                           crs='EPSG:4326'
)
argo_gdf = gpd.GeoDataFrame(argo_df, 
                            geometry=gpd.points_from_xy(argo_df.lon, argo_df.lat),
                            crs='EPSG:4326'
)

To view the relative locations of ICESat-2 and Argo, the below cell uses the `explore()` function from GeoPandas. For large datasets like ICESat-2, loading the map might take a while.

In [ ]:
# Plot ICESat-2 track (medium/high confidence photons only) on a map
m = is2_gdf[is2_gdf['signal_conf']>=3].explore(tiles='Esri.WorldImagery',
                                             name='ICESat-2')

# Add Argo float locations to map
argo_gdf.explore(m=m, name='Argo', marker_kwds={"radius": 6}, color='red')

While we're at it, let's plot temperature and pressure profiles for each of the Argo floats in the area.

In [ ]:
# Plot vertical profile of temperature vs. pressure for all of the floats
fig, ax = plt.subplots(figsize=(12, 6))
for pid in np.unique(argo_df['profile_id']):
    argo_df[argo_df['profile_id']==pid].plot(ax=ax, x='temperature', y='pressure', label=pid)
plt.gca().invert_yaxis()
plt.xlabel('Temperature [$\degree$C]')
plt.ylabel('Pressure [hPa]')
plt.ylim([750, -10])
plt.tight_layout()

Lastly, let's look at some near-coincident ICESat-2 and Argo data in a multi-panel plot.

In [ ]:
# Only consider ICESat-2 signal photons
is2_pd_signal = is2_pd[is2_pd['signal_conf']>0]

## Multi-panel plot showing ICESat-2 and Argo data

# Calculate Extent
lons = [-154, -143, -143, -154, -154]
lats = [30, 30, 37, 37, 30]
lon_margin = (max(lons) - min(lons)) * 0.1
lat_margin = (max(lats) - min(lats)) * 0.1

# Create Plot
fig,([ax1,ax2],[ax3,ax4]) = plt.subplots(2, 2, figsize=(12, 6))

# Plot Relative Global View
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.plot(ax=ax1, color='0.8', edgecolor='black')
argo_df.plot.scatter(ax=ax1, x='lon', y='lat', s=25.0, c='green', zorder=3, alpha=0.3)
is2_pd.plot.scatter(ax=ax1, x='lon', y='lat', s=10.0, zorder=2, alpha=0.3)
ax1.plot(lons, lats, linewidth=1.5, color='orange', zorder=2)
#df.plot(ax=ax2, x='lon', y='lat', marker='o', color='red', markersize=2.5, zorder=3)
ax1.set_xlim(-160,-100)
ax1.set_ylim(20,50)
ax1.set_aspect('equal', adjustable='box')
ax1.set_xlabel('Longitude', fontsize=18)
ax1.set_ylabel('Latitude', fontsize=18)

# Plot Zoomed View of Ground Tracks
argo_df.plot.scatter(ax=ax2, x='lon', y='lat', s=50.0, c='green', zorder=3, alpha=0.3)
is2_pd.plot.scatter(ax=ax2, x='lon', y='lat', s=10.0, zorder=2, alpha=0.3)
ax2.plot(lons, lats, linewidth=1.5, color='orange', zorder=1)
ax2.scatter(-151.98956, 34.43885, color='orange', marker='^', s=80, zorder=4)
ax2.set_xlim(min(lons) - lon_margin, max(lons) + lon_margin)
ax2.set_ylim(min(lats) - lat_margin, max(lats) + lat_margin)
ax2.set_aspect('equal', adjustable='box')
ax2.set_xlabel('Longitude', fontsize=18)
ax2.set_ylabel('Latitude', fontsize=18)

# Plot ICESat-2 along-track vertical profile. A dotted line notes the location of a nearby Argo float
is2 = ax3.scatter(is2_pd_signal['lat'], is2_pd_signal['height'], s=0.1)
ax3.axvline(34.43885, linestyle='--', linewidth=3, color='black')
ax3.set_xlim([34.3, 34.5])
ax3.set_ylim([-15, 5])
ax3.set_xlabel('Latitude', fontsize=18)
ax3.set_ylabel('Approx. IS-2 Depth [m]', fontsize=16)
ax3.set_yticklabels(['15', '10', '5', '0', '-5'])

# Plot vertical ocean profile of the nearby Argo float
argo_df[argo_df['profile_id']=='4903409_053'].plot(ax=ax4, x='temperature', y='pressure', linewidth=3)
ax4.set_yscale('log')
ax4.invert_yaxis()
ax4.get_legend().remove()
ax4.set_xlabel('Temperature [$\degree$C]', fontsize=18)
ax4.set_ylabel('Argo Pressure', fontsize=16)

plt.tight_layout()

# Save figure
#plt.savefig('/home/jovyan/icepyx/is2_argo_figure.png', dpi=500)